In [1]:
from pathlib import Path
from copy import deepcopy
import os
import zarr
import dask.array as da
from functools import partial
import qtpy

import numpy as np
import pandas as pd
import tifffile
from ipywidgets import IntSlider, VBox
import fastplotlib as fpl

from caiman.motion_correction import high_pass_filter_space
from caiman.summary_images import correlation_pnr

os.environ['WAYLAND_DISPLAY'] = ''
os.environ['RUST_LOG'] = 'info'
os.environ['WINIT_UNIX_BACKEND'] = 'x11'

import mesmerize_core as mc

ImportError: cannot import name 'select_adapter' from 'pygfx.renderers.wgpu' (/home/mbo/miniconda3/envs/mbo-caiman/lib/python3.11/site-packages/pygfx/renderers/wgpu/__init__.py)

In [2]:
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

In [3]:
parent_path = Path().home() / "caiman_data_org"
movie_path = parent_path / 'animal_01' / 'session_01' / 'plane_1.zarr'

batch_path = parent_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(parent_path))

# you could alos load the registration batch and
# save this patch in a new dataframe (saved to disk automatically)
try:
    df = mc.load_batch(batch_path)
except (IsADirectoryError, FileNotFoundError):
    df = mc.create_batch(batch_path)

df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,plane_1,animal_01/session_01/plane_1.zarr,"{'main': {'var_name_hdf5': 'mov', 'max_shifts'...",{'mean-projection-path': '247620be-d14b-46da-b...,2024-09-09T14:55:09,2024-09-10T09:57:59,42.13 sec,None,247620be-d14b-46da-b6ba-6c73bf607b02
1,cnmf,caiman_phys,247620be-d14b-46da-b6ba-6c73bf607b02/247620be-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 'bee3b92b-c567-4c7c-b...,2024-09-10T12:50:48,2024-09-10T12:52:31,98.29 sec,None,bee3b92b-c567-4c7c-bb29-d460e08fa980
2,cnmf,caiman_phys,247620be-d14b-46da-b6ba-6c73bf607b02/247620be-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",None,2024-09-12T17:17:46,None,None,None,a2043cde-7b58-49a8-8bbc-376c690af125


In [4]:
def read_zarr(path):
    # return zarr.open(path)['mov']
    return da.from_zarr(path, 'mov').compute()

interactive(children=(Text(value='/home/mbo/caiman_data_org', continuous_update=False, description='parent_pat…